In [14]:
import json
import pandas as pd
import glob
from collections import defaultdict
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from tokenizers import Tokenizer
from transformers import pipeline
import warnings
import os
import torch

## Ko-Bart 

사투리 -> 표준어 번역 프로세싱

In [2]:
dir = "/Users/bykwon/Desktop/daiv/24w/labels" #경상도 텍스트 데이터

print(dir)

files = glob.glob(f"{dir}/*.json")
df = pd.DataFrame(index=range(2), columns=["사투리","표준어"])

for file in files:
    # print(file)
    _dict = defaultdict(list)
    
    with open(file) as f:
        json_data = json.load(f)
        # JSON 파일의 utterance 키 값 아래에 리스트형태로 데이터가 존재했다.
        data = json_data.get("utterance")
        
        for item in data:
            # standard_form(표준어)와 dialect_form(사투리)가 일치하지 않는다면?
            standard = item.get("standard_form")
            dialect = item.get("dialect_form")
            if standard != dialect:
                _dict["표준어"].append(standard)
                _dict["사투리"].append(dialect)
    
    # 한 JSON 파일에서 데이터 추출이 끝나면 비어있던 데이터프레임에 추가
    append_df = pd.DataFrame(_dict)
    df = pd.concat([df, append_df])

df.to_csv("data.tsv",sep="\t")

/Users/bykwon/Desktop/daiv/24w/labels


In [3]:
len(df)

336375

## 전처리

In [4]:
import re

step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')

#예시 문장
text1 = "&name2& 님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?"
text2 = "어~ {반가움} &name123&! (()) 반갑고 ㅋㅋ -오- 오랜만이다"

# 무식해보이지만 일단 확인해보자!
for text in [text1,text2]:
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)
    
    print(f"Text : {text}")
    print(f"Step 1 : {after_step1}")
    print(f"Step 2 : {after_step2}")
    print(f"Step 3 : {after_step3}")
    print(f"Step 4 : {after_step4}")
    print("\n\n")

Text : &name2& 님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 1 :  님은 뭐~ {laughing} (()) 힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 2 :  님은 뭐~ {laughing}  힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 3 :  님은 뭐~   힘들 때 -위- 위로가 되어 준 노래 같은 게 있으신가요?
Step 4 :  님은 뭐~   힘들 때  위로가 되어 준 노래 같은 게 있으신가요?



Text : 어~ {반가움} &name123&! (()) 반갑고 ㅋㅋ -오- 오랜만이다
Step 1 : 어~ {반가움} ! (()) 반갑고 ㅋㅋ -오- 오랜만이다
Step 2 : 어~ {반가움} !  반갑고 ㅋㅋ -오- 오랜만이다
Step 3 : 어~  !  반갑고 ㅋㅋ -오- 오랜만이다
Step 4 : 어~  !  반갑고 ㅋㅋ  오랜만이다





## 데이터 정제

In [5]:
df = pd.read_csv(f'data.tsv',sep='\t')

# data.tsv를 불러왔더니 nan 값이 들어있어 일단 삭제해줬다.
df = df.dropna()

# 아까 지정해뒀던 정규표현식들
step1 = re.compile(r'&\w+&')
step2 = re.compile(r'\(*\)')
step3 = re.compile(r'\{[^}]*\}')
step4 = re.compile(r'\-[^}]*\-')


def make_cleaned_text(text):
    after_step1=step1.sub('',text)
    after_step2=step2.sub('',after_step1)
    after_step3=step3.sub('',after_step2)
    after_step4=step4.sub('',after_step3)

    return after_step4

for idx, row in df.iterrows():
    df.at[idx, '사투리'] = make_cleaned_text(row['사투리'])
    df.at[idx, '표준어'] = make_cleaned_text(row['표준어'])
    
df = df.iloc[:, 1:]

df.to_csv('data_fixed.tsv', index=False, sep='\t')

In [6]:
df.tail()

,사투리,표준어
336370,얘네들이 이케 새끼를 낳고 나면은 이제 그때,얘네들이 이렇게 새끼를 낳고 나면은 이제 그때
336371,한 마리는 저기 경북 어디에서 구조되고 이렇게 해서 다 따른 시기에 육 개월 이상 ...,한 마리는 저기 경북 어디에서 구조되고 이렇게 해서 다 다른 시기에 육 개월 이상 ...
336372,그까 얘네들을 개 농장에서 만들어서,그러니까 얘네들을 개 농장에서 만들어서
336373,왜냐면 너무 까 말씀하신 것처럼 자기가 키우고 싶어서 데꼬 가놓곤,왜냐면 너무 까 말씀하신 것처럼 자기가 키우고 싶어서 데리고 가놓곤
336374,어느 시점에 이제 더 키우기 싫어져서 막 버리는 건 쫌 잘못됐다라고 생각을 해서,어느 시점에 이제 더 키우기 싫어져서 막 버리는 건 조금 잘못됐다라고 생각을 해서
